In [98]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
# Gather training and testing data and process it to be configured for time intervals
from common import *
from sklearn.model_selection import train_test_split
import pandas as pd

nba = get_cleaned_external_data()
# nba = get_cleaned_baseline_data()
# nba = add_log_y_values(nba)
# X_train, X_test, y_train, y_test = split_data(nba, time_based_split=False)

X = nba.loc[:, nba.columns != 'inflationAdjSalary']
y = nba.loc[:, nba.columns == 'inflationAdjSalary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

print(y_test)


[[2490716]
 [3253148]
 [3433524]
 ...
 [5835848]
 [1666846]
 [ 142610]]


In [100]:
# print(nba.columns)
# y_test = np.exp(y_test)
# y_train = np.exp(y_train)
# print(y_test)
# nba = nba.drop(['inflationAdjSalary'], axis=1)

def pad_y(y) :
    # y = y[:,np.newaxis]
    tmp = np.zeros((y.shape[0],len(nba.columns)))
    tmp[:,len(nba.columns)-1:] = y
    return tmp

def pad_x(x) :
    tmp = np.zeros((x.shape[0],len(nba.columns)))
    tmp[:,:len(nba.columns)-1] = x
    return tmp

def unpad_x(x) :
    return x[:,:len(nba.columns)-1]

def unpad_y(y) :
    return y[:,len(nba.columns)-1:]


# Scale the stuff down
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(nba)
# X_train = unpad_x(scaler.transform(pad_x(X_train)))
# X_test = unpad_x(scaler.transform(pad_x(X_test)))
# y_train = unpad_y(scaler.transform(pad_y(y_train)))
# y_test = unpad_y(scaler.transform(pad_y(y_test)))

# print(y_train_scaled)
# print(X_train_scaled[0,:])
# Sanity check the scaler
# print((nba['FG']).iloc[0])
# print(X_train_scaled[0, X.columns.get_loc('FG')])
# inv = scaler.inverse_transform(X_scaled)
# print(inv[0, X.columns.get_loc('FG')])


# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size = 0.25, random_state=42)

In [101]:
# Fit the model
from sklearn.neural_network import MLPRegressor
import numpy as np

# MLPClassifier only classifies data as integers or strings, therefore, our problem is one of regression for the neural network
# Consequently I should use the mlp regressor
mlp = MLPRegressor(hidden_layer_sizes=(20,10,10), solver='adam', max_iter=1000)
mlp.fit(X_train,y_train)

c:\Users\mmart\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\mmart\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(20, 10, 10), max_iter=1000)

In [103]:
# Tune the Hyerparameters
from sklearn.model_selection import RandomizedSearchCV
# Define the hyperparameters you want to tune
params = {
    'hidden_layer_sizes': [(10,), (20,20,30), (5,5,5,5), (20,10,5), (10,10,10), (20,10,10)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.1]
}

# Use GridSearchCV to search over the hyperparameter grid
rs = RandomizedSearchCV(mlp, param_distributions=params, n_iter=120, cv=5, n_jobs=-1, random_state=42)

# Fit the GridSearchCV object on the training data
rs.fit(X_train, y_train)

# Print the best set of hyperparameters
print("Best Hyperparameters:", rs.best_params_)

# Evaluate the model on the validation set using the best hyperparameters
mlp = rs.best_estimator_

c:\Users\mmart\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 108 is smaller than n_iter=120. Running 108 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\mmart\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Hyperparameters: {'learning_rate_init': 0.01, 'hidden_layer_sizes': (20, 20, 30), 'alpha': 0.0001, 'activation': 'relu'}


In [ ]:
# Evaluate the model
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
predict_test = mlp.predict(X_test)
test_set_rsquared = mlp.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(np.log(predict_test), np.log(y_test)))
test_set_mse = mean_squared_error(np.log(predict_test), np.log(y_test))
print('R_squared value: ', r2_score(np.log(y_test), np.log(predict_test))) # Variances are not nicely correlated
print('RMSE: ', test_set_rmse) # Fairly good at data prediction
print('MSE:', test_set_mse)

R_squared value:  0.3840255276664155
RMSE:  0.8893512001208703
MSE: 0.7909455571564323


In [ ]:
# # Denormalize the data
# predict_test = mlp.predict((X_scaled[0]).reshape(1,-1))
# print(X_train.shape[1])
# print(predict_test)
# tmp = np.zeros((1,X_train.shape[1]))
# tmp[:,:predict_test.shape[1]] = predict_test
# print(tmp)
# undone = scaler.inverse_transform(tmp)
# print(undone.flatten().tolist()[:13])
# print(y[base_feats].iloc[0])

NameError: name 'X_scaled' is not defined

In [ ]:
# # Compute the confidence interval of each of the features
# from common import *

# # Get the unscaled predictions
# predict_full = mlp.predict(X_scaled)
# tmp = np.zeros(X_scaled.shape)
# tmp[:,:predict_full.shape[1]] = predict_full
# predicted_unscaled_vals = scaler.inverse_transform(tmp)
# predicted_unscaled_vals = predicted_unscaled_vals[:,:predict_full.shape[1]]

# # compute the RMSE for each feature in the output vector
# actual_unscaled_vals = y[base_feats].values
# rmse = np.sqrt(np.square(predicted_unscaled_vals - actual_unscaled_vals))

# # compute the percentilse of the RMSE to get the confidence interval of each predicted feature
# ci_up = np.percentile(rmse, 95, axis=0) # 95th percentile
# ci_low = np.percentile(rmse, 5, axis=0) # 5th percentile
# print("Off by at most this much:", ci_up.flatten().tolist())
# print(np.max(actual_unscaled_vals, axis=0))
# print(np.min(actual_unscaled_vals, axis=0)) 
# # print(ci_low.flatten().tolist()) 

# # TODO: validate these findings with the very basic network to see if it predicts that these stats will land us in our desired class
# np.set_printoptions(suppress=True, precision=4)
# print(confidence_interval_numpy(predicted_unscaled_vals, actual_unscaled_vals)) # requires the assumption of gaussianity, but seems to be working?


# print(bootstrap_confidence_interval(predicted_unscaled_vals, actual_unscaled_vals)) # should be able to work without the assumption of gaussianity
# # computing for rmse, so says that the true root mean squared error has a 95% of landing in these intervals
# # residuals are likely far smaller because you have the negatives balancing things out so it is going to be less accurate for computing the confidence interval

# residuals = predicted_unscaled_vals - actual_unscaled_vals
# ci = np.quantile(residuals, 1 - 0.05, axis=0)
# print("HELLO")
# print(ci)

# # I want to say that my prediction is between these 2 bounds with 95% confidence
# # The two functions I provided seem to find the rmse of each parameter with 95% confidence
# # the first and 4th method I try do not scale the mean of the data and therefore, I don't think they necessarily apply to giving me the confidence interval I desire



# # DO A CASE STUDY on an individual
# # ANALYZE THIS MODEL ON BASELINE AND EXTERNAL




Off by at most this much: [1162.6010485438574, 564.5294819959297, 0.9358547475622839, 36.678941065626624, 44.77237141825249, 5.681788518771645, 0.20083569593628162, 176.79757130821858, 50.42570265081218, 279.9271334943758, 140.38598344515225, 67.25290821346142, 211.85565399097388, 2.097975557900953, 17.64002385748879]
[3485.  2832.    40.    85.    83.    45.3    6.   925.   225.  1226.
  756.   402.   978.    90.   325. ]
[  1.    0.   19.    1.    0.  -40.7   0.    0.    0.    0.    0.    0.
   0.   63.  135. ]
[[476.1579 496.692 ]
 [212.2525 222.3542]
 [  0.3576   0.3772]
 [ 13.7566  14.3724]
 [ 18.2243  18.9838]
 [  2.1591   2.2802]
 [  0.072    0.0783]
 [ 56.3487  59.6562]
 [ 18.633   19.5375]
 [ 99.8705 104.9828]
 [ 48.5911  51.2492]
 [ 21.9521  23.1666]
 [ 80.2483  84.0312]
 [  0.7928   0.831 ]
 [  7.0573   7.3689]]
{'column_0': (475.79476645622674, 496.67383016962964), 'column_1': (212.41596990296983, 222.70172029234558), 'column_2': (0.35803130884824563, 0.3767257665685465), '

In [ ]:
# from sklearn.ensemble import RandomForestRegressor 
# random_forest_model = RandomForestRegressor(n_estimators = 60, random_state = 0)

# random_forest_model.fit(X_train, y_train) 

# y_pred = random_forest_model.predict(X_test)

# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("MSE: ",mse)
# print("RMSE: ",rmse)
